### Curso cientista de dados - Projeto final

Sobre o projeto: Uma empresa de empréstimos fictícia sofre com a inadimpência de seus clientes e o nosso objetivo é reduzir essa taxa.

Criar um modelo de machine learning a partir de um banco de dados


In [1]:
#Importação de bibliotecas
import pandas as pd
import numpy as np
import statistics  as sts
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from sklearn.compose import make_column_transformer
import tensorflow as tf
import psycopg2


Conectando o banco de dados ao python

In [2]:
#conexão com o banco de dados
conexao = psycopg2.connect(host="localhost",database="Projeto", user="postgres", password="12345", port=5432)

In [3]:
#cria um cursor
cursor = conexao.cursor()
#define consulta
consulta = "select * from credito"
#executa
cursor.execute(consulta)

In [4]:
#recupera todos os registros
registros = cursor.fetchall()

In [ ]:
#percorre os registros e imprime
#for registro in registros:
#    print(registro)

Tratamento dos dados

In [6]:
# a variavel registro nos vem como uma tupla, então transformamos em array usando o numpy
db = pd.DataFrame(np.array(registros))

In [7]:
#Damos nomes para as colunas através do comando columns do pandas
db.columns = ["IDCREDITO", "Duracao", "HistoricoCredito", "Proposito", "Valor", "Investimentos", "Emprego", "TempoParcelamento", "EstadoCivil", 
              "FiadorTerceiros", "ResidenciaDesde", "Idade", "OutrosFinanciamentos", "Habitacao", "EmprestimoExistente", "Profissao", 
              "Dependentes", "SocioEmpresa", "Estrangeiro", "Status"]
db.head()

,IDCREDITO,Duracao,HistoricoCredito,Proposito,Valor,Investimentos,Emprego,TempoParcelamento,EstadoCivil,FiadorTerceiros,ResidenciaDesde,Idade,OutrosFinanciamentos,Habitacao,EmprestimoExistente,Profissao,Dependentes,SocioEmpresa,Estrangeiro,Status
0,2,48,2,1,5951,2,3,2,1,3,2,22,3,1,1,4,1,0,0,0
1,3,12,1,2,2096,2,4,2,4,3,3,49,3,1,1,3,2,0,0,1
2,4,42,2,3,7882,2,4,2,4,4,4,45,3,2,1,4,2,0,0,1
3,5,24,3,4,4870,2,3,3,4,3,4,53,3,2,2,4,2,0,0,0
4,6,36,2,2,9055,1,3,2,4,3,4,35,3,2,1,3,2,1,0,1


In [409]:
#verificamos o formato do banco de dados para ver se está ok e podemos começar a explorar os dados
db.shape

(1000, 20)

In [410]:
#Verificar se existem valores nulos(NaN) na database
db.isnull().sum()

IDCREDITO                0
Duracao                  0
HistoricoCredito         0
Proposito                0
Valor                    0
Investimentos            0
Emprego                 10
TempoParcelamento        0
EstadoCivil              0
FiadorTerceiros          0
ResidenciaDesde          7
Idade                    0
OutrosFinanciamentos     0
Habitacao                9
EmprestimoExistente      0
Profissao                0
Dependentes              0
SocioEmpresa             0
Estrangeiro              0
Status                   0
dtype: int64

In [411]:
#A coluna emprego possui um valor NaN, mas como não é um  valor numérico, utilizamos a Moda
#Repetimos o processo para as colunas ResidenciaDesde e Habitacao
emprego = db.groupby(['Emprego']).size()
emprego

Emprego
1    171
2    247
3    338
4    172
5     62
dtype: int64

In [412]:
#Subistituir o valor NaN pela moda
db['Emprego'].fillna(3, inplace = True)

In [413]:
#Repetimos o processo para a coluna ResidenciaDesde
residencia = db.groupby(['ResidenciaDesde']).size()
residencia

ResidenciaDesde
1    129
2    308
3    147
4    409
dtype: int64

In [414]:
#Subistituir o valor NaN pela moda
db['ResidenciaDesde'].fillna(4, inplace = True)

In [415]:
#Repetimos o processo para a coluna Habitacao
habitacao = db.groupby(['Habitacao']).size()
habitacao

Habitacao
1    709
2    107
3    175
dtype: int64

In [416]:
#Subistituir o valor NaN pela moda
db['Habitacao'].fillna(1, inplace = True)

In [417]:
#Repetimos o processo para a coluna Profissao, pois analizando o banco de dados descobrimos um valor não nulo, porém fora dos padrões
profissao = db.groupby(['Profissao']).size()
profissao

Profissao
1      147
2       22
3      197
4      625
999      9
dtype: int64

In [418]:
#Como o valor 999 não é nulo, precisamos localizá-lo na tabela e substituílo pela moda.
db.loc[(db['Profissao'] > 5 ), 'Profissao'] = 4

In [419]:
profissao = db.groupby(['Profissao']).size()
profissao

Profissao
1    147
2     22
3    197
4    634
dtype: int64

In [420]:
db = db.drop(columns=['IDCREDITO', 'Valor', 'Duracao', 'Proposito', ])

In [421]:
db.head()

,HistoricoCredito,Investimentos,Emprego,TempoParcelamento,EstadoCivil,FiadorTerceiros,ResidenciaDesde,Idade,OutrosFinanciamentos,Habitacao,EmprestimoExistente,Profissao,Dependentes,SocioEmpresa,Estrangeiro,Status
0,2,2,3,2,1,3,2,22,3,1,1,4,1,0,0,0
1,1,2,4,2,4,3,3,49,3,1,1,3,2,0,0,1
2,2,2,4,2,4,4,4,45,3,2,1,4,2,0,0,1
3,3,2,3,3,4,3,4,53,3,2,2,4,2,0,0,0
4,2,1,3,2,4,3,4,35,3,2,1,3,2,1,0,1


Criação e treinamento do modelo

In [422]:
x= db.iloc[:,0:15].values #previsores
y= db.iloc[:,15].values #classe

In [423]:
LE = LabelEncoder()
scaler = StandardScaler()

In [424]:
for z in range(15):
    x[:,z] = LE.fit_transform(x[:,z])
x.shape

(1000, 15)

In [425]:
y = LE.fit_transform(y)
y

array([0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0,
       0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1,
       0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0,
       0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1,

In [426]:
# Divisão da base em treino e teste (70% para treinamento e 30% para teste)
x_treinamento, x_teste, y_treinamento, y_teste = train_test_split(x, y, 
                                                                  test_size=0.3,
                                                                  random_state=0)

In [427]:
# Criação da estrutura da rede neural com a classe Sequential (sequência de camadas)
modelo = Sequential()
modelo.add(Dense(units = 30, input_dim = 15))
modelo.add(Dense(units = 38))
modelo.add(Dense(units = 27, activation = 'softmax'))

In [428]:
x_treinamento =  np.asarray(x_treinamento).astype(np.int64)
y_treinamento =  np.asarray(y_treinamento).astype(np.int64)
x_treinamento = tf.convert_to_tensor(x_treinamento, dtype=tf.int64)
y_treinamento = tf.convert_to_tensor(y_treinamento, dtype=tf.int64)
x_teste = tf.convert_to_tensor(x_treinamento, dtype=tf.int64)
y_teste = tf.convert_to_tensor(y_treinamento, dtype=tf.int64)

In [429]:
modelo.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

modelo.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_33 (Dense)            (None, 30)                480       
                                                                 
 dense_34 (Dense)            (None, 38)                1178      
                                                                 
 dense_35 (Dense)            (None, 27)                1053      
                                                                 
Total params: 2,711
Trainable params: 2,711
Non-trainable params: 0
_________________________________________________________________


In [430]:
modelo.fit(x_treinamento, y_treinamento, epochs = 100, validation_data = [x_teste, y_teste])

Epoch 1/100
22/22 [==============================] - 0s 6ms/step - loss: 3.1958 - accuracy: 0.3600 - val_loss: 1.1993 - val_accuracy: 0.6614
Epoch 2/100
22/22 [==============================] - 0s 2ms/step - loss: 0.8787 - accuracy: 0.6286 - val_loss: 0.7694 - val_accuracy: 0.6500
Epoch 3/100
22/22 [==============================] - 0s 2ms/step - loss: 0.7217 - accuracy: 0.6314 - val_loss: 0.6814 - val_accuracy: 0.6443
Epoch 4/100
22/22 [==============================] - 0s 2ms/step - loss: 0.6710 - accuracy: 0.6414 - val_loss: 0.6679 - val_accuracy: 0.6614
Epoch 5/100
22/22 [==============================] - 0s 2ms/step - loss: 0.6770 - accuracy: 0.6571 - val_loss: 0.6377 - val_accuracy: 0.6557
Epoch 6/100
22/22 [==============================] - 0s 2ms/step - loss: 0.6369 - accuracy: 0.6643 - val_loss: 0.6365 - val_accuracy: 0.6500
Epoch 7/100
22/22 [==============================] - 0s 2ms/step - loss: 0.6204 - accuracy: 0.6600 - val_loss: 0.6099 - val_accuracy: 0.6914
Epoch 8/100
2